In [ ]:
"""
This notebook is about detecting rice seedling from the input image.
@author Hsin-Yi Yang
@acknowledgement All of the images used in the research were provided by GEOSAT Aerospace & Technology Inc.
"""

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import csv

from skimage import morphology
from skimage.future import graph
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte, img_as_float
from skimage.color import label2rgb

from scipy import ndimage as ndi
from skimage.morphology import disk
from skimage.filters import rank

def imgResize(image,dsRatio):
    dsImg = cv2.resize(image, (int(len(image[0])*dsRatio), int(len(image)*dsRatio)), interpolation=cv2.INTER_AREA)
    return dsImg

def normalize(matrix):
    matrix = 255 * (matrix - matrix.min()) / (matrix.max() - matrix.min())
    matrix = matrix.astype(np.uint8)
    return matrix

def complement(img):
    return 255-img

def calcTGI(B, G, R):
    blue = img_as_float(B)
    green = img_as_float(G)
    red = img_as_float(R)
    TGI = green - 0.39*red - 0.61*blue
    TGI = normalize(TGI)
    return TGI

def calcVARI(B, G, R):
    blue = img_as_float(B)
    green = img_as_float(G)
    red = img_as_float(R)
    fraction = green - red
    denominator = (green - red)/(green + red - blue)
    VARI = np.divide(fraction, denominator, out=np.zeros_like(green), where=denominator!=0)
    VARI = normalize(VARI)
    return VARI    

In [ ]:
"""label with rag"""
inputpath = "./input/"
inputfiles = [file for file in os.listdir(inputpath) if file.endswith('.JPG')]
inputfiles.sort()
outputpath = "./output/"

datasetpath = "./dataset/mask/"
datasetfiles = [file for file in os.listdir(datasetpath) if file.endswith('.jpg')]
datasetfiles.sort()
datasetpath_img = "./dataset/img/"

print("preparing for dataset")
start = time.time()
dataset = []

# calculate the Hue and Saturation histogram of the dataset images in HSV color space
for i in range(len(datasetfiles)):
    label, name = datasetfiles[i].split(".")[0].split("_")
    img = cv2.imread(datasetpath_img + name + ".JPG")
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.imread(datasetpath + datasetfiles[i], cv2.IMREAD_GRAYSCALE)
    hist = cv2.calcHist([img_hsv], [0,1], mask, [180,256], [0,180,0,256])
#     hist = cv2.calcHist([img], [0,1,2], mask, [256,256,256], [0,256,0,256,0,256])
    cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX);
    info = {"label": label, "hist": hist}
    dataset.append(info)

end = time.time()
print("preparing time:", end-start)

VI = [{"vi": "Gray", "thresh": 4}, 
      {"vi": "CLAHE", "thresh": 9}, 
      {"vi": "TGI", "thresh": 4}, 
      {"vi": "VARI", "thresh": 2}]

vi_seedlings = []
img_area = 6000*4000

for i in range(len(inputfiles)):
    start = time.time()
    img = cv2.imread(inputpath + inputfiles[i])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    name = inputfiles[i][:8]
    print("input:", name)  

    ## generating the preprocessing images
    vi_start = time.time()
    IMG = []

    # preprocessing image 1: convert the image into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # preprocessing image 2: enhance the grayscale image using CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit = 5) 
    Gray = clahe.apply(gray)

    B, G, R = cv2.split(img)
    # preprocessing image 3: compute the vegetation index image by TGI (Triangular Greenness Index)
    TGI = calcTGI(B, G, R)
    TGI = complement(TGI)
    # preprocessing image 4: compute the vegetation index image by VARI (Visible Atmospherically Resistant Index)
    VARI = calcVARI(B, G, R)

    IMG.append(gray)
    IMG.append(Gray)
    IMG.append(TGI)
    IMG.append(VARI)

    vi_end = time.time()
    print("Generating preprocessing imgs, time:", vi_end-vi_start)   
    
    ## generating the superpixel image by SLIC (Simple Linear Iterative Clustering)
    start_slic = time.time()
    img_rgb = cv2.medianBlur(img_rgb, 5)
    segments_slic = slic(img_rgb, n_segments = 1500, compactness = 15)
    slic_id = np.unique(segments_slic)
    print("# of superpixels:", len(slic_id))
    end_slic = time.time()

    # coloring each superpixel on the image
    colorimg = label2rgb(segments_slic, image=img, alpha=0.5)
    colorimg = mark_boundaries(colorimg, segments_slic, color=(0, 1, 1), outline_color=(0, 1, 1))
    colorimg = img_as_ubyte(colorimg)
    cv2.imwrite(outputpath + "slic/" + name + "_n1500_color.jpg", colorimg)
    print("SLIC time:", end_slic-start_slic)

    # By computing the RAG (Region Adjacency Graph) using mean colors,    
    # if the mean colors of neighboring superpixels are closed enough to each other, the superpixels will be merged into one.
    thresh = 15
    print("start merging the superpixels if the mean colors between them are closed enough.")
    start_rag = time.time()
    g = graph.rag_mean_color(img_rgb, segments_slic)
    segments_slic2 = graph.cut_threshold(segments_slic, g, thresh)
    slic_id2 = np.unique(segments_slic2)
    print("# of superpixels after rag thresholding:", len(slic_id2))  
    
    # after merging step, coloring each superpixel on the image
    colorimg = label2rgb(segments_slic2, image=img, alpha=0.5)
    colorimg = mark_boundaries(colorimg, segments_slic2, color=(0, 1, 1), outline_color=(0, 1, 1))
    colorimg = img_as_ubyte(colorimg)
    cv2.imwrite(outputpath + "slic_merge/" + name + "_n1500_color_rag_t" + str(thresh) +".jpg", colorimg)
    end_rag = time.time()
    print("merging time:", end_rag-start_rag)
    
    ## labeling each superpixel as paddy or not paddy with 3-NN classification
    mask_paddy = np.zeros(img.shape[:2], dtype = "uint8")
    print("start extracting paddy field.")
    start_label = time.time()
    for index in slic_id2:
        area = np.count_nonzero(segments_slic2 == index)
        # calculating the proportion of the superpixel area on the img area
        proportion = area / img_area
        if proportion < 0.01:
            continue
        
        slic_ID = name + "_" + str(index)
        slic_mask = np.zeros(img.shape[:2], dtype = "uint8")
        slic_mask[segments_slic2 == index] = 255

        hist_slic = cv2.calcHist([img_hsv], [0,1], slic_mask, [180,256], [0,180,0,256])
        cv2.normalize(hist_slic, hist_slic, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX);
        # checking the superpixel is the most similar wiht which label the dataset image is.
        comparehist = []
        for data in dataset:
            correl = cv2.compareHist(hist_slic, data["hist"], cv2.HISTCMP_CORREL)
            info = {"label": data["label"], "correl": correl}
            comparehist.append(info)
            
        compare = sorted(comparehist, key = lambda item : item["correl"], reverse = True)
        paddy_label = 0
        for i in range(3):
            if compare[i]["correl"] < 0:
                continue
            if compare[i]["label"]=="paddy":
                paddy_label += 1

        # if paddy label is more than 1 in the labels with top 3 highest correlation, 
        # than the superpixel will be considered as paddy field.
        # The final mask_paddy is the mask of paddy field.
        if paddy_label > 1:
            mask_paddy = cv2.bitwise_or(mask_paddy, slic_mask)
    
    end_label = time.time()
    print("label time:", end_label-start_label)

    # morphology on the mask_paddy
    test = time.time()
    kernel = np.ones((45,45),np.uint8)
    opening = cv2.morphologyEx(mask_paddy, cv2.MORPH_OPEN, kernel, iterations=5)
    mask_convex = morphology.convex_hull_object(opening)
    mask_convex = 255*mask_convex
    mask_convex = mask_convex.astype(np.uint8)
    test = time.time() - test
    print("morphology time:", test)

    paddy = cv2.bitwise_and(img, img, mask = mask_convex)

    # save the mask image
    cv2.imwrite(outputpath + "paddy/" + name + "_paddy.jpg", paddy)
    cv2.imwrite(outputpath + "paddy_mask/" + name + "_mask.jpg", mask_convex)
    mask = mask_convex

    ## applying blob detection on the preprocessing images with mask_paddy
    print("start blob detection")
    for item in VI:
        seedlings = []
        vid = VI.index(item)
        vi = item["vi"]
        img_vi = IMG[vid]
        
        cv2.imwrite(outputpath + "preprocess/" + name + "_" + vi + ".jpg", img_vi)
        
        paddy_vi = cv2.bitwise_and(img_vi, img_vi, mask = mask)
        
        thresh = item["thresh"]
        print(vi, "threshStep =", thresh)
        
        # Set up the detector with default parameters.
        parameters = cv2.SimpleBlobDetector_Params()
        # We detect the darker blobers, so the ExG image should be inversed
        #darker blobers: blobColor = 0; lighter blobers: blobColor=255  
        parameters.filterByColor = True
        parameters.blobColor = 0
        # setting minArea  = 100 will filter out all the blobs that have less then 100 pixels
        parameters.filterByArea = True
        parameters.minArea = 15
        parameters.filterByCircularity = True
        parameters.minCircularity = 0.1
        parameters.filterByConvexity = True
        parameters.minConvexity = 0.87
        parameters.thresholdStep = thresh
        
        # setting the blob detector
        detector = cv2.SimpleBlobDetector_create(parameters)
        
        start_blob = time.time()

        keypoints_blob = detector.detect(paddy_vi)
        paddy_blob = img.copy()
        paddy_blob = cv2.drawKeypoints(paddy_blob, keypoints_blob, np.array([]), (255,0,0), v2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        kps_blob = len(keypoints_blob)
        print("kps_" + vi + "=", kps_blob)
        
        cv2.imwrite(outputpath + "seedlings/img/" + name + "_" + vi + "_blob.jpg", paddy_blob)
        info = {'fileName': name, 'VI': vi, 'keypoints': kps_blob}
        vi_seedlings.append(info)
        
        for keypt in keypoints_blob:
            info = {'keypoints': str(keypt.pt)}
            seedlings.append(info)
        
        ## write the positions of the keypoints in a csv file
        csvpath = outputpath + "seedlings/position/"
        csvname = name + "_" + vi + '.csv'
        print("write", csvname)
        with open(csvpath + csvname, 'w', newline='') as csvfile:
            fieldnames = ['keypoints']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for i in seedlings:
                writer.writerow(i)
        
        end_blob = time.time()
        print(vi + " blob time:", end_blob-start_blob)
        

## record the quantities of detected seedlings in each detection result
with open(outputpath + "seedlings/seedlings.csv", 'w', newline='') as csvfile:
    fieldnames = ['fileName', 'VI', 'keypoints']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in vi_seedlings:
        writer.writerow(i)    